In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

#Imports of functions from other notebooks
from ipynb.fs.full.Leaderboard import position_at_last_mark
from ipynb.fs.full.DELTA import reachmark_delta
from ipynb.fs.full.race_timestamps import start_and_end_of_start
from ipynb.fs.full.sequence import start_sequence


pd.set_option("max_rows", None)
pd.set_option("max_columns", None)

In [5]:
df1 = pd.read_csv('2022-09-10/log_DEN_2022-09-10_1s_renamed.csv')
# df2 = pd.read_csv('2022-05-15/log_USA_2022-05-15_1s_renamed.csv') #TRK_RACE_NUM_unk	
# df3 = pd.read_csv('2022-06-18/log_USA_2022-06-18_1s_renamed.csv')
# df4 = pd.read_csv('2022-06-19/log_USA_2022-06-19_1s_renamed.csv')
# df5 = pd.read_csv('2022-07-30/log_USA_2022-07-30_1s_renamed.csv')
# df6 = pd.read_csv('2022-07-31/log_USA_2022-07-31_1s_renamed.csv')
# df7 = pd.read_csv('2022-08-20/log_USA_2022-08-20_1s_renamed.csv')
# df8 = pd.read_csv('2022-09-10/log_USA_2022-09-10_1s_renamed.csv')
# df9 = pd.read_csv('2022-09-11/log_USA_2022-09-11_1s_renamed.csv')
# df10 = pd.read_csv('2022-09-24/log_USA_2022-09-24_1s_renamed.csv')
# df11 = pd.read_csv('2022-09-25/log_USA_2022-09-25_1s_renamed.csv') #TRK_RACE_NUM_unk, TRK_LEG_NUM_unk

# dfer  = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11]

# # "{}/log_{}_{}_1s_renamed.csv"
# # BDE_RACE_NUM_unk
# # BDE_RACE_NUM_unk

In [6]:
features = ['BOAT','DATETIME','Latitude','Longitude',
            'BSP_kmh','TWS_kmh','TWD','TWD_TM_deg','TWA',
            'TRK_RACE_NUM_unk','TRK_LEG_NUM_unk',
            'PC_TTB_s','PC_TTK_s','TTS_s']


# Loading of races 

In [7]:
from os.path import exists as file_exists

def load_races(countries, races):
    """ This functions loads the races
    for each day and each country. 
    Does the feature selection. 
    Does the datacleaning. 
    Calc the new variables. 
    Add all dataframes together 
    Input: .csv files
    -------------------------------
    Output: dataframe 
    """

    list_of_csv_file_names = []
    df_of_daily_dataframes = pd.DataFrame()
    df_of_dataframe = pd.DataFrame()
    done_data = pd.DataFrame()
    delta_df = pd.DataFrame()
    for date in races:
        
        for countryname in countries:
            
            csv_file = "{}/log_{}_{}_1s_renamed.csv".format(date, countryname,date)
            if (file_exists(csv_file) == False): #Checking if file excist
                print(csv_file, 'does not excist')
            if file_exists(csv_file): #Checking if file excist
                list_of_csv_file_names.append(csv_file)
                df = pd.read_csv(csv_file)
                for i in range(len(df.columns)):
                    if df.columns[i]=='BDE_RACE_NUM_unk':
                        df.rename(columns = {'BDE_RACE_NUM_unk':'TRK_RACE_NUM_unk'}, inplace = True)
                    if df.columns[i]=='BDE_LEG_NUM_unk':
                        df.rename(columns = {'BDE_LEG_NUM_unk':'TRK_LEG_NUM_unk'}, inplace = True)
                
                
                df = df[features]
                df_of_daily_dataframes = df_of_daily_dataframes.append(df,ignore_index=True)

    
##-------- Now we have all data from all boats for one day. The delta calculation is made:
            
        # convert the 'Date' column to datetime format
        df_of_daily_dataframes['DATETIME']= pd.to_datetime(df_of_daily_dataframes['DATETIME'])
        #Sort by DateTime
        df_of_daily_dataframes = df_of_daily_dataframes.sort_values(by="DATETIME").reset_index(drop=True)
        df_of_daily_dataframes["TRK_LEG_NUM_unk"]=df_of_daily_dataframes["TRK_LEG_NUM_unk"].astype(float)
        df_of_daily_dataframes["TRK_RACE_NUM_unk"]=df_of_daily_dataframes["TRK_RACE_NUM_unk"].astype(float)
        
##---------Defining start and end times-------------------->>
        start_times = start_and_end_of_start(df_of_daily_dataframes)
# ##---------Define dataframe interval--------------------##
        df_interval_start = start_sequence(df_of_daily_dataframes)


##---------Creating leaderboard:------------------------##
        leaderboard = position_at_last_mark(df_interval_start, start_times)
    

        
##---------Creating delta times at reach mark: --------##
        delta = reachmark_delta(leaderboard, start_times)
    

##---------Creating the dataframe: --------------------## 
        df_of_dataframe = df_of_dataframe.append(delta,ignore_index=True)

        #Now emptying that day ready for a new day
        df_of_daily_dataframes= pd.DataFrame()
        
        

    return(df_of_dataframe)

In [8]:
#All countries
count=('DEN', 'USA','NZL','CAN','GBR','AUS','ESP','FRA','SUI') 
#All racedays
# '2022-05-14','2022-05-15', Skipping Bermuda since there is TWD_TM_deg
rac=('2022-06-18','2022-06-19','2022-07-30','2022-07-31','2022-08-20','2022-09-10','2022-09-11',
     '2022-09-24','2022-09-25','2022-11-12','2022-11-13')

DATAAA = load_races(count,rac)


2022-08-20/log_GBR_2022-08-20_1s_renamed.csv does not excist


In [9]:
DATAAA = DATAAA.loc[DATAAA['TRK_LEG_NUM_unk'].isin([0,1,2,3,4,5,6,7,8])]

In [10]:
df = DATAAA
df = df[df.TRK_RACE_NUM_unk !=0]
df['DATETIME']= pd.to_datetime(df['DATETIME'])
df.to_csv('df.csv',index =False)

C:\Users\katja\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
